In [10]:
import zipfile
import os

zip_file_path = "/content/archive.zip"
extraction_dir = "/content/plant_disease_dataset"

# Create the extraction directory if it doesn't exist
os.makedirs(extraction_dir, exist_ok=True)

In [11]:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

## Load and preprocess the data


In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_dir = "/content/plant_disease_dataset"

In [13]:
image_size = (128, 128)
batch_size = 32
validation_split = 0.2

datagen = ImageDataGenerator(rescale=1./255, validation_split=validation_split)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 33022 images belonging to 2 classes.
Found 8254 images belonging to 2 classes.


## Build the cnn model

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,898 (12.61 MB)

 Trainable params: 3,304,898 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

## Train the model

In [15]:
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1032/1032 ━━━━━━━━━━━━━━━━━━━━ 631s 610ms/step - accuracy: 0.4977 - loss: 0.7007 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 634s 614ms/step - accuracy: 0.4977 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 684s 616ms/step - accuracy: 0.4986 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 641s 621ms/step - accuracy: 0.4969 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 5/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 635s 615ms/step - accuracy: 0.4980 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6933
Epoch 6/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 635s 615ms/step - accuracy: 0.5016 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 7/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 668s 648ms/step - accuracy: 0.4987 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 8/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 634s 614ms/step - accuracy: 0.5

## Evaluate the model


In [18]:
loss, accuracy = model.evaluate(validation_generator)

print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

258/258 ━━━━━━━━━━━━━━━━━━━━ 48s 184ms/step - accuracy: 0.4997 - loss: 0.6931
Validation Loss: 0.6931
Validation Accuracy: 0.5000


Executive Summary: Plant Leaf Disease Detection Project Using Deep Learning

This project aims to develop a deep learning model to detect plant leaf diseases and classify them as "healthy" or "infected" using leaf image data. A dataset containing various images of plant leaves was used.

Methodology:

Data Extraction: Leaf images were extracted from a zip file.

Data Loading and Preprocessing: ImageDataGenerator from TensorFlow/Keras was used to load images, resize them to 128 x 128, and normalize pixel values. The data was automatically split into two sets, with a training and validation ratio of 80% and validation ratio of 20%.

CNN Model Construction: A convolutional neural network (CNN) model was built, consisting of Conv2D, MaxPooling2D, Flatten, and Dense layers. The "relu" activation function was used in the hidden layers, and "softmax" was used in the final output layer for binary classification. The model was compiled using the Adam optimizer, the Cross-Classification loss function, and the accuracy metric.

Model Training: The model was trained for ten epochs using training and validation generators.

Model Evaluation: The model's performance was evaluated on the validation set.

Results:

Evaluation of the model on the validation set showed the following results:

Validation Loss: {{loss:.4f}}
Validation Accuracy: {{accuracy:.4f}}

Conclusion and Recommendations:

The evaluation results indicate that the current model achieved a modest accuracy of {{accuracy:.4f}} on the validation data. This suggests that the model may not effectively distinguish between healthy and infected leaves based on the current training.

To improve model performance, the following steps are recommended:

Increase the number of training epochs to enable the model to learn better from the data.

Modify the model architecture, such as adding additional layers or changing filter sizes, to capture more complex features.

Use regularization techniques such as Dropout to prevent overfitting.
Experiment with different learning rates for the optimizer.
Explore data augmentation techniques to increase the size and diversity of the training dataset.
Analyze the dataset to understand the class distribution and image quality.
This executive summary provides an overview of the project and its initial results. Improving performance will require further iteration and experimentation with the model architecture and training parameters.